In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import gc
from itertools import combinations
import warnings
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from warnings import simplefilter
import joblib
import os
import matplotlib.pyplot as plot

In [2]:
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

is_train = True
is_infer = False

train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
train = train.dropna(subset=['target'])

def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
    
    # auction has two parts: https://nasdaqtrader.com/content/ETFs/closing_cross_faqs.pdf
    df["auction_stage"] = np.where(df["seconds_in_bucket"] > 300, 1, 0)
        
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        'date_id', 
    ], inplace=True)
        
    gc.collect()
    
    return df

# order by date
# train_date_sort = train.sort_values(by=["date_id"])
trainset = train

y_all = trainset['target'].values
X_all = feat_eng(trainset.drop(columns='target'))

y_min = np.min(trainset["target"].values)
y_max = np.max(trainset["target"].values)

In [3]:
print(y_min, y_max)

-385.2898 446.07043


In [4]:
# X_ = X_all[np.isfinite(y_all)]
# y_ = y_all[np.isfinite(y_all)]
# print(X_all.shape, X_.shape)

# print(f"The number of train data: {train.shape}")
# print(X_all.isnull().values.any())
# print(X_.isnull().values.any())

In [5]:
N_Folds = 4
kf = KFold(n_splits=N_Folds, shuffle=True, random_state=42)
# kf = KFold(n_splits=N_Folds, shuffle=False)
mae_scores = []

params = {
    'learning_rate': 0.018,
    'max_depth': 10,
    'iterations': 1500,
#     'max_leaves': 440,
    'objective': 'MAE',
    'random_seed': 42,
#     'reg_alpha': 0.01,
    'reg_lambda': 0.01
}

is_train = True
if is_train:
#     X = X_all[X_all["date_id"] >= 240]
#     y = y_all[X_all["date_id"] >= 240]
    X = X_all
    y = y_all
    for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
        print(train_idx)
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

#         m = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=50, early_stopping_rounds=50)
        m = cat.CatBoostRegressor(**params)
        m.fit(X_train, y_train, 
                    eval_set=[(X_valid, y_valid)], 
                    verbose=50, 
                    early_stopping_rounds=50
                    )
#         m = xgb.train(params, train_data, evals=[valid_data],callbacks=[eval_log,early_stop])
        print(f"Fold {fold+1} Trainning finished.")

        model_filename = f"./model_fold_{fold+1}.pkl"
        joblib.dump(m, model_filename)
        y_pred_valid = m.predict(X_valid)

        y_pred_valid = np.nan_to_num(y_pred_valid)
        y_valid = np.nan_to_num(y_valid)
        mae = mean_absolute_error(y_valid, y_pred_valid)
        mae_scores.append(mae)

    # 计算五折平均的MAE
    average_mae = np.mean(mae_scores)
    print(f"5 fold MAE: {average_mae}")


In [6]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out

is_infer = False
if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    predictions = []

    for (test, revealed_targets, sample_prediction) in iter_test:
        feat = feat_eng(test)
        fold_prediction = 0
        for fold in range(0, N_Folds):
            model_filename = f"/kaggle/input/catboost15kit/model_fold_{fold+1}.pkl"
            m = joblib.load(model_filename)
            fold_prediction += m.predict(feat)   
        
        fold_prediction /= N_Folds
        fold_prediction = zero_sum(fold_prediction, test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
        clipped_predictions = np.clip(fold_prediction, y_min, y_max)
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
